In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv(r"D:\Machine Learning\final_preprocessed_data.csv")

In [2]:
df.head()

,state_code,dist_code,type_name,purpose_name,disp_name,Judge_Labels,Lag,Total_Days
0,1,1,1943,2975,33,55,0.0,37.0
1,1,1,1943,3315,52,55,0.0,31.0
2,1,1,1943,5877,52,55,0.0,31.0
3,1,1,1943,840,52,55,0.0,31.0
4,1,1,1943,840,5,55,0.0,8.0


In [3]:
df['type_name'].value_counts()

981     513845
977     426036
736     328940
1892    294604
853     275986
         ...  
5390      2071
3090      2046
5297      1922
4975      1572
4947      1036
Name: type_name, Length: 500, dtype: int64

In [4]:
top_100_positions = df['type_name'].value_counts().head(100).index
df = df[df['type_name'].isin(top_100_positions)]

In [5]:
df.shape

(9295134, 8)

In [6]:
top_40_positions = df['Judge_Labels'].value_counts().head(40).index
df = df[df['Judge_Labels'].isin(top_40_positions)]

In [7]:
df.shape

(8980039, 8)

In [8]:
df[df['type_name']==981]['Total_Days'].value_counts()

0.0       157671
1.0         5783
2.0         3434
3.0         2450
7.0         2425
           ...  
1159.0         1
1186.0         1
1300.0         1
1315.0         1
1148.0         1
Name: Total_Days, Length: 1164, dtype: int64

In [9]:
import pandas as pd

for value in df['type_name'].unique():
    specific_type_name = value
    specific_type_data = df[df['type_name'] == specific_type_name]
    Q1 = specific_type_data['Total_Days'].quantile(0.25)
    Q3 = specific_type_data['Total_Days'].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.3 * IQR
    upper_bound = Q3 + 1.3 * IQR
    df = df[(df['type_name'] != specific_type_name) | ((df['type_name'] == specific_type_name) & (df['Total_Days'] >= lower_bound) & (df['Total_Days'] <= upper_bound))]


In [10]:
df.shape

(7977613, 8)

In [11]:
df.head()

,state_code,dist_code,type_name,purpose_name,disp_name,Judge_Labels,Lag,Total_Days
0,1,1,1943,2975,33,55,0.0,37.0
1,1,1,1943,3315,52,55,0.0,31.0
2,1,1,1943,5877,52,55,0.0,31.0
3,1,1,1943,840,52,55,0.0,31.0
4,1,1,1943,840,5,55,0.0,8.0


In [13]:
bins = [-1,40,150,250 ,350, float('inf')]
labels = [1, 2, 3, 4,5]


df['total_Days_Group'] = pd.cut(df['Total_Days'], bins=bins, labels=labels)
df['total_Days_Group'] = df['total_Days_Group'].astype('int')


In [14]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [15]:
X = df[['state_code','dist_code', 'type_name','Judge_Labels']]
y = df['total_Days_Group']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


In [16]:
best_params = {
    'objective': 'regression',
    'num_leaves': 80,
    'n_estimators': 600,
    'learning_rate': 0.3,
    'feature_fraction': 0.8,
    'boosting_type': 'gbdt',
    'bagging_freq': 4,
    'bagging_fraction': 0.8,
    'force_col_wise':True
}

train_data = lgb.Dataset(X_train, label=y_train)

In [17]:
model = lgb.train(params=best_params, train_set=train_data, num_boost_round=200)

y_pred = model.predict(X_test)

C:\Users\dell\anaconda3\lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Info] Total Bins 246
[LightGBM] [Info] Number of data points in the train set: 6382090, number of used features: 4
[LightGBM] [Info] Start training from score 1.496618


In [18]:
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

Mean Squared Error: 0.529732967913147
R-squared: 0.49334611996957156


In [19]:
for i in range(len(y_pred)):
    if y_pred[i] < 0:
        y_pred[i] = 1
    elif y_pred[i] >= 0 and y_pred[i] < 1:
        y_pred[i] = 1
    elif y_pred[i] >= 1 and y_pred[i] < 1.5:
        y_pred[i] = 1
    elif y_pred[i] >= 1.5 and y_pred[i] < 2:
        y_pred[i] = 2
    elif y_pred[i] >= 2 and y_pred[i] < 2.5:
        y_pred[i] = 2
    elif y_pred[i] >= 2.5 and y_pred[i] < 3:
        y_pred[i] = 3
    elif y_pred[i] >= 3 and y_pred[i] < 3.5:
        y_pred[i] = 3
    elif y_pred[i] >= 3.5 and y_pred[i] < 4:
        y_pred[i] = 4
    elif y_pred[i] >= 4 and y_pred[i] <= 4.5:
        y_pred[i] = 4
    elif y_pred[i] > 4.5:
        y_pred[i] = 5

In [20]:
from sklearn.metrics import accuracy_score
acc = accuracy_score(y_test,y_pred)
acc

0.7061678208336702

In [21]:
pred = pd.DataFrame(y_pred)
pred.value_counts()

1.0    1083107
2.0     343607
3.0     120519
4.0      46650
5.0       1640
dtype: int64

In [22]:
y_test.value_counts()

1    1187443
2     203402
3      87706
5      65347
4      51625
Name: total_Days_Group, dtype: int64

In [23]:
import pickle
with open('model_days_grp.pkl','wb') as f:
    pickle.dump(model,f)

In [28]:
df['type_name'].value_counts().index.sort_values()

Int64Index([ 284,  720,  721,  723,  724,  734,  736,  763,  853,  856,  977,
             981, 1016, 1104, 1105, 1523, 1532, 1548, 1554, 1702, 1706, 1778,
            1803, 1809, 1810, 1820, 1822, 1892, 1896, 1937, 1938, 1943, 2035,
            2045, 2064, 2073, 2082, 2090, 2243, 2264, 2281, 2283, 2284, 2290,
            2299, 2302, 2303, 2327, 2338, 2355, 2639, 2660, 2920, 2934, 3231,
            3247, 3692, 3694, 3850, 4051, 4484, 4746, 4782, 4784, 4823, 4826,
            4875, 4882, 4918, 4920, 5003, 5032, 5070, 5085, 5098, 5158, 5188,
            5539, 5559, 5593, 5671, 5707, 5943, 5979, 6049, 6716, 6743, 6748,
            6769, 7014, 7038, 7043, 7048, 7065, 7261, 7295, 7300, 7333, 7375,
            7405],
           dtype='int64')

In [244]:
df.head()

,state_code,dist_code,type_name,purpose_name,disp_name,Judge_Labels,Lag,Total_Days,total_Days_Group,Lag_grp
0,1,1,1943,2975,33,55,0.0,37.0,1,1
1,1,1,1943,3315,52,55,0.0,31.0,1,1
2,1,1,1943,5877,52,55,0.0,31.0,1,1
3,1,1,1943,840,52,55,0.0,31.0,1,1
4,1,1,1943,840,5,55,0.0,8.0,1,1


In [248]:
df['Lag'].value_counts()

0.0       3201440
1.0        792305
2.0        400959
3.0        250269
4.0        193858
           ...   
1176.0          1
1094.0          1
1267.0          1
991.0           1
1095.0          1
Name: Lag, Length: 1109, dtype: int64

In [268]:
bins1 = [-1,10, 30, 80, 200, float('inf')]
labels1 = [1, 2, 3, 4,5]


df['Lag_grp'] = df['Lag_grp'].astype('int')
df['Lag_grp']= pd.cut(df['Lag'],bins = bins1, labels = labels1)

In [251]:
df['Lag_grp'].value_counts()

1    7092122
2     559740
3     235965
4      76172
5      13614
Name: Lag_grp, dtype: int64

In [275]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score


X_lag = df[['state_code','dist_code', 'type_name','Judge_Labels']]
y_lag = df['Lag_grp']

X_train_lag, X_test_lag, y_train_lag, y_test_lag = train_test_split(X_lag, y_lag, test_size=0.2)


best_params = {
    'objective': 'regression',
    'num_leaves': 80,
    'n_estimators': 600,
    'learning_rate': 0.3,
    'feature_fraction': 0.8,
    'boosting_type': 'gbdt',
    'bagging_freq': 4,
    'bagging_fraction': 0.8,
    'force_col_wise':True
}


train_data = lgb.Dataset(X_train_lag, label=y_train_lag)

model_lag = lgb.train(params=best_params, train_set=train_data, num_boost_round=200)

y_pred_lag = model_lag.predict(X_test_lag)

mse = mean_squared_error(y_test_lag, y_pred_lag)
r2 = r2_score(y_test_lag, y_pred_lag)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")


C:\Users\dell\anaconda3\lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Info] Total Bins 247
[LightGBM] [Info] Number of data points in the train set: 6382090, number of used features: 4
[LightGBM] [Info] Start training from score 1.164825
Mean Squared Error: 0.1728771210646099
R-squared: 0.3703487422043864


In [276]:
y_pred = y_pred_lag
for i in range(len(y_pred)):
    if y_pred_lag[i] < 0:
        y_pred[i] = 1
    elif y_pred[i] >= 0 and y_pred[i] < 1:
        y_pred[i] = 1
    elif y_pred[i] >= 1 and y_pred[i] < 1.5:
        y_pred[i] = 1
    elif y_pred[i] >= 1.5 and y_pred[i] < 2:
        y_pred[i] = 2
    elif y_pred[i] >= 2 and y_pred[i] < 2.5:
        y_pred[i] = 2
    elif y_pred[i] >= 2.5 and y_pred[i] < 3:
        y_pred[i] = 3
    elif y_pred[i] >= 3 and y_pred[i] < 3.5:
        y_pred[i] = 3
    elif y_pred[i] >= 3.5 and y_pred[i] < 4:
        y_pred[i] = 4
    elif y_pred[i] >= 4 and y_pred[i] <= 4.5:
        y_pred[i] = 4
    elif y_pred[i] > 4.5:
        y_pred[i] = 5

In [277]:
from sklearn.metrics import accuracy_score
acc = accuracy_score(y_test_lag,y_pred_lag)
acc

0.8779923573649518

In [278]:
predi = pd.DataFrame(y_pred)
predi.value_counts()

1.0    1422351
2.0     161905
3.0       9429
4.0       1620
5.0        218
dtype: int64

In [279]:
y_test_lag.value_counts()

1    1418632
2     111840
3      46986
4      15357
5       2708
Name: Lag_grp, dtype: int64

In [280]:
import pickle
with open('Court_model_lag_last_updated.pkl','wb') as f:
    pickle.dump(model_lag,f)